In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/store-sales-time-series-forecasting/oil.csv
/kaggle/input/store-sales-time-series-forecasting/sample_submission.csv
/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv
/kaggle/input/store-sales-time-series-forecasting/stores.csv
/kaggle/input/store-sales-time-series-forecasting/train.csv
/kaggle/input/store-sales-time-series-forecasting/test.csv
/kaggle/input/store-sales-time-series-forecasting/transactions.csv


In [2]:
# 한글 폰트 다운
import os
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

plt.rc('font', family='NanumBarunGothic') 
def change_matplotlib_font(font_download_url):
    FONT_PATH = 'MY_FONT'
    
    font_download_cmd = f"wget {font_download_url} -O {FONT_PATH}.zip"
    unzip_cmd = f"unzip -o {FONT_PATH}.zip -d {FONT_PATH}"
    os.system(font_download_cmd)
    os.system(unzip_cmd)
    
    font_files = fm.findSystemFonts(fontpaths=FONT_PATH)
    for font_file in font_files:
        fm.fontManager.addfont(font_file)

    font_name = fm.FontProperties(fname=font_files[0]).get_name()
    matplotlib.rc('font', family=font_name)
    print("font family: ", plt.rcParams['font.family'])

font_download_url = "https://fonts.google.com/download?family=Noto%20Sans%20KR"
change_matplotlib_font(font_download_url)

--2024-03-27 03:55:33--  https://fonts.google.com/download?family=Noto%20Sans%20KR
Resolving fonts.google.com (fonts.google.com)... 142.251.170.139, 142.251.170.138, 142.251.170.102, ...
Connecting to fonts.google.com (fonts.google.com)|142.251.170.139|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: 'MY_FONT.zip'

     0K .......... .......... .......... .......... .......... 29.2M
    50K .......... .......... .......... .......... .......... 15.1M
   100K .......... .......... .......... .......... .......... 18.3M
   150K .......... .......... .......... .......... .......... 27.8M
   200K .......... .......... .......... .......... .......... 29.1M
   250K .......... .......... .......... .......... .......... 29.6M
   300K .......... .......... .......... .......... .......... 57.5M
   350K .......... .......... .......... .......... .......... 60.5M
   400K .......... .......... .......... .......... .......... 40

Archive:  MY_FONT.zip
  inflating: MY_FONT/OFL.txt         
  inflating: MY_FONT/static/NotoSansKR-Thin.ttf  
  inflating: MY_FONT/static/NotoSansKR-ExtraLight.ttf  
  inflating: MY_FONT/static/NotoSansKR-Light.ttf  
  inflating: MY_FONT/static/NotoSansKR-Regular.ttf  
  inflating: MY_FONT/static/NotoSansKR-Medium.ttf  
  inflating: MY_FONT/static/NotoSansKR-SemiBold.ttf  
  inflating: MY_FONT/static/NotoSansKR-Bold.ttf  
  inflating: MY_FONT/static/NotoSansKR-ExtraBold.ttf  
  inflating: MY_FONT/static/NotoSansKR-Black.ttf  
  inflating: MY_FONT/NotoSansKR-VariableFont_wght.ttf  
  inflating: MY_FONT/README.txt      
font family:  ['Noto Sans KR']


In [3]:
import matplotlib.pyplot as plt
import seaborn as sns

In [155]:
oil = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/oil.csv")
sample_submission = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/sample_submission.csv")
holidays_events = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv")
stores = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/stores.csv")
train = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/train.csv")
test = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/test.csv")
transactions = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/transactions.csv")

In [156]:
# 시간 별 정리
train['date'] = pd.to_datetime(train['date'])
train['sales'] = np.log1p(train['sales'])

test['date'] = pd.to_datetime(test['date'])

transactions['date'] = pd.to_datetime(transactions['date'])

holidays_events['date'] = pd.to_datetime(holidays_events['date'])

oil['dcoilwtico'].fillna(method='backfill', inplace=True)
# oil['dcoilwtico'] = oil['dcoilwtico'].interpolate(method='linear')
# oil['dcoilwtico'][0] = 93.14
oil['date'] = pd.to_datetime(oil['date'])

/tmp/ipykernel_33/4236478000.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  oil['dcoilwtico'].fillna(method='backfill', inplace=True)
/tmp/ipykernel_33/4236478000.py:11: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  oil['dcoilwtico'].fillna(method='backfill', inplace=True)


In [157]:
df = pd.concat([train, test], axis=0)

df = pd.merge(df, stores, how='left', on='store_nbr')
df = pd.merge(df, transactions, how='left', on=['date', 'store_nbr'])
df = pd.merge(df, holidays_events, how='left', on='date')
df = pd.merge(df, oil, how='left', on='date')

In [99]:
def fill_nan(df):
    df['type_y'] = df['type_y'].fillna('Common')
    df['locale'] = df['locale'].fillna('Common')
    df['description'] = df['description'].fillna('Unknown')
    df['transferred'] = df['transferred'].fillna(False)
    df['dcoilwtico'] = df['dcoilwtico'].fillna(method='backfill')
    return df

def create_date_features(df):
    df['month'] = df['date'].dt.month
    df['day_of_month'] = df['date'].dt.day
#     df['day_of_year'] = df['date'].dt.dayofyear
#     df['week_of_year'] = df['date'].dt.isocalendar().week
    df['day_of_week'] = df['date'].dt.dayofweek
    df['year'] = df['date'].dt.year
    return df

In [158]:
df = fill_nan(df)

/tmp/ipykernel_33/4198969709.py:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['transferred'] = df['transferred'].fillna(False)
/tmp/ipykernel_33/4198969709.py:6: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['dcoilwtico'] = df['dcoilwtico'].fillna(method='backfill')


In [69]:
df.isna().sum()

id                    0
date                  0
store_nbr             0
family                0
sales             28512
onpromotion           0
city                  0
state                 0
type_x                0
cluster               0
transactions     277629
type_y                0
locale                0
locale_name     2578554
description           0
transferred           0
dcoilwtico            0
dtype: int64

In [159]:
df = create_date_features(df)

In [71]:
df.isna().sum()

id                    0
date                  0
store_nbr             0
family                0
sales             28512
onpromotion           0
city                  0
state                 0
type_x                0
cluster               0
transactions     277629
type_y                0
locale                0
locale_name     2578554
description           0
transferred           0
dcoilwtico            0
month                 0
day_of_month          0
day_of_week           0
year                  0
dtype: int64

In [72]:
df

,id,date,store_nbr,family,sales,onpromotion,city,state,type_x,cluster,...,type_y,locale,locale_name,description,transferred,dcoilwtico,month,day_of_month,day_of_week,year
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13,...,Holiday,National,Ecuador,Primer dia del ano,False,93.14,1,1,1,2013
1,1,2013-01-01,1,BABY CARE,0.0,0,Quito,Pichincha,D,13,...,Holiday,National,Ecuador,Primer dia del ano,False,93.14,1,1,1,2013
2,2,2013-01-01,1,BEAUTY,0.0,0,Quito,Pichincha,D,13,...,Holiday,National,Ecuador,Primer dia del ano,False,93.14,1,1,1,2013
3,3,2013-01-01,1,BEVERAGES,0.0,0,Quito,Pichincha,D,13,...,Holiday,National,Ecuador,Primer dia del ano,False,93.14,1,1,1,2013
4,4,2013-01-01,1,BOOKS,0.0,0,Quito,Pichincha,D,13,...,Holiday,National,Ecuador,Primer dia del ano,False,93.14,1,1,1,2013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3082855,3029395,2017-08-31,9,POULTRY,NaN,1,Quito,Pichincha,B,6,...,Common,Common,NaN,Unknown,False,47.26,8,31,3,2017
3082856,3029396,2017-08-31,9,PREPARED FOODS,NaN,0,Quito,Pichincha,B,6,...,Common,Common,NaN,Unknown,False,47.26,8,31,3,2017
3082857,3029397,2017-08-31,9,PRODUCE,NaN,1,Quito,Pichincha,B,6,...,Common,Common,NaN,Unknown,False,47.26,8,31,3,2017
3082858,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,NaN,9,Quito,Pichincha,B,6,...,Common,Common,NaN,Unknown,False,47.26,8,31,3,2017


In [73]:
df['day_of_week'].value_counts()

day_of_week
6    445500
3    443718
1    440154
2    440154
5    440154
4    436590
0    436590
Name: count, dtype: int64

주말 / 평일 처리 해줄까?

In [28]:
# lag_features(df, lags=[*range(1, 16), 16, 17, 18, 19, 20, 21, 22, 30, 31, 90, 180, 364])

,id,date,store_nbr,family,sales,onpromotion,city,state,type_x,cluster,...,sales_t-18,sales_t-19,sales_t-20,sales_t-21,sales_t-22,sales_t-30,sales_t-31,sales_t-90,sales_t-180,sales_t-364
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2013-01-01,1,BABY CARE,0.0,0,Quito,Pichincha,D,13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2013-01-01,1,BEAUTY,0.0,0,Quito,Pichincha,D,13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2013-01-01,1,BEVERAGES,0.0,0,Quito,Pichincha,D,13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,2013-01-01,1,BOOKS,0.0,0,Quito,Pichincha,D,13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3082855,3029395,2017-08-31,9,POULTRY,NaN,1,Quito,Pichincha,B,6,...,6.024556,5.951650,6.265727,5.679561,5.811536,6.347732,6.155947,6.313937,5.657400,5.917013
3082856,3029396,2017-08-31,9,PREPARED FOODS,NaN,0,Quito,Pichincha,B,6,...,4.665032,4.874464,4.728272,4.726768,4.824025,3.940863,4.142182,4.347358,4.551210,4.828018
3082857,3029397,2017-08-31,9,PRODUCE,NaN,1,Quito,Pichincha,B,6,...,7.435206,7.258598,7.282127,6.944510,7.178887,7.812565,7.325513,7.430288,7.049694,7.768148
3082858,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,NaN,9,Quito,Pichincha,B,6,...,5.303305,4.934474,4.948760,5.003946,4.430817,5.318120,4.043051,1.791759,2.079442,4.382027


In [26]:
# def lag_features(df, lags):
#     for lag in lags:
#         df[f"sales_t-{lag}"] = df.groupby(["store_nbr", "family"])["sales"].transform(
#             lambda x: x.shift(lag))
#     return df

In [ ]:
# def roll_mean_features(dataframe, windows):
#     def add_noise(x):
#         return x + np.random.normal(size=len(x))

#     for window in windows:
#         dataframe['sales_roll_mean_' + str(window)] = dataframe.groupby(["store_nbr", "family"])['sales']. \
#                                                           transform(
#             lambda x: x.shift(16).rolling(window=window, min_periods=7, win_type="triang").mean())
#         dataframe['sales_roll_mean_' + str(window)] = dataframe.groupby(["store_nbr", "family"])['sales_roll_mean_' + str(window)].transform(add_noise)
#     return dataframe

In [ ]:
# def ewm_features(dataframe, alphas, lags):
#     for alpha in alphas:
#         for lag in lags:
#             feature_name = 'sales_ewm_alpha_' + str(alpha).replace(".", "") + '_lag_' + str(lag)
#             dataframe[feature_name] = dataframe.groupby(["store_nbr", "family"])['sales'].\
#                 transform(lambda x: x.shift(lag).ewm(alpha=alpha, min_periods=1).mean())
#     return dataframe

In [89]:
df.columns

Index(['id', 'date', 'store_nbr', 'family', 'sales', 'onpromotion', 'city',
       'state', 'type_x', 'cluster', 'transactions', 'type_y', 'locale',
       'locale_name', 'description', 'transferred', 'dcoilwtico', 'month',
       'day_of_month', 'day_of_week', 'year'],
      dtype='object')

In [160]:
col = ['date','store_nbr', 'family', 'sales', 'onpromotion', 'cluster', 'type_y', 'locale', 'description', 'transferred','dcoilwtico']
df = df[col]

In [105]:
# df['store_nbr'] = df['store_nbr'].astype('category')
# df['family'] = df['family'].astype('category')
# df['store_nbr'] = df['store_nbr'].astype('category')
# df['cluster'] = df['cluster'].astype('category')
# df['family'] = df['family'].astype('category')
# df['type_y'] = df['type_y'].astype('category')
# df['locale'] = df['locale'].astype('category')
# df['description'] = df['description'].astype('category')

In [161]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df['family'] = label_encoder.fit_transform(df['family'])
df['type_y'] = label_encoder.fit_transform(df['type_y'])
df['locale'] = label_encoder.fit_transform(df['locale'])
df['description'] = label_encoder.fit_transform(df['description'])
df['transferred'] = label_encoder.fit_transform(df['transferred'])

In [139]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2425302 entries, 657558 to 3082859
Data columns (total 11 columns):
 #   Column       Dtype         
---  ------       -----         
 0   date         datetime64[ns]
 1   store_nbr    int64         
 2   family       int64         
 3   sales        float64       
 4   onpromotion  int64         
 5   cluster      int64         
 6   type_y       int64         
 7   locale       int64         
 8   description  int64         
 9   transferred  int64         
 10  dcoilwtico   float64       
dtypes: datetime64[ns](1), float64(2), int64(8)
memory usage: 222.0 MB


In [162]:
df = df[df['date'] > '2013-12-31']
x = df[df['date'] <= '2017-08-15'].drop('date', axis=1)
test_x = df[df['date'] > '2017-08-15'].drop('date', axis=1)

In [148]:
x

,store_nbr,family,sales,onpromotion,cluster,type_y,locale,description,transferred,dcoilwtico
657558,1,0,0.000000,0,13,4,2,50,0,95.14
657559,1,1,0.000000,0,13,4,2,50,0,95.14
657560,1,2,0.000000,0,13,4,2,50,0,95.14
657561,1,3,0.000000,0,13,4,2,50,0,95.14
657562,1,4,0.000000,0,13,4,2,50,0,95.14
...,...,...,...,...,...,...,...,...,...,...
3054343,9,28,6.084802,0,6,4,1,28,0,47.57
3054344,9,29,5.046987,1,6,4,1,28,0,47.57
3054345,9,30,7.791824,148,6,4,1,28,0,47.57
3054346,9,31,4.804021,8,6,4,1,28,0,47.57


In [163]:
corr = x.corr()
corr['sales'].sort_values(ascending=False)

sales          1.000000
onpromotion    0.358750
cluster        0.038165
locale         0.007074
store_nbr     -0.001648
description   -0.002076
transferred   -0.006218
type_y        -0.009086
family        -0.056900
dcoilwtico    -0.118022
Name: sales, dtype: float64

In [164]:
y = x['sales']
x = x.drop(['sales'], axis=1)
test_x = test_x.drop(['sales'], axis=1)

In [165]:
x.shape, y.shape, test_x.shape

((2396790, 9), (2396790,), (28512, 9))

In [171]:
test_x

,store_nbr,family,onpromotion,cluster,type_y,locale,description,transferred,dcoilwtico
3054348,1,0,0,13,2,0,100,0,46.80
3054349,1,1,0,13,2,0,100,0,46.80
3054350,1,2,2,13,2,0,100,0,46.80
3054351,1,3,20,13,2,0,100,0,46.80
3054352,1,4,0,13,2,0,100,0,46.80
...,...,...,...,...,...,...,...,...,...
3082855,9,28,1,6,2,0,100,0,47.26
3082856,9,29,0,6,2,0,100,0,47.26
3082857,9,30,1,6,2,0,100,0,47.26
3082858,9,31,9,6,2,0,100,0,47.26


# model

In [187]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

def train_linear_regressor(X, y):
    # 학습 및 테스트 데이터로 분할
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # 선형 회귀 모델 생성 및 학습
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    # 학습된 모델 평가
    train_pred = model.predict(X_train)
    test_pred = model.predict(X_test)
    
    # 평가 지표 계산
    train_rmse = mean_squared_error(y_train, train_pred, squared=False)
    test_rmse = mean_squared_error(y_test, test_pred, squared=False)
    
    print("Train RMSE:", train_rmse)
    print("Test RMSE:", test_rmse)
    
    return model

# X는 입력 특성(feature), y는 목표 변수(target)입니다.
# X와 y는 각각 특성 행렬(feature matrix)과 타겟 벡터(target vector)여야 합니다.
# 이 예제에서는 X_train과 y_train이 주어졌다고 가정합니다.
# X_train, y_train은 모델을 학습하기 위한 훈련 데이터입니다.
# X_test, y_test는 모델을 평가하기 위한 테스트 데이터입니다.

# 모델 훈련 예시:
model = train_linear_regressor(x, y)

# 모델로 예측하는 예시:
predictions = model.predict(test_x)


KeyboardInterrupt: 

In [179]:
test_x

,store_nbr,family,onpromotion,cluster,type_y,locale,description,transferred,dcoilwtico
3054348,1,0,0,13,2,0,100,0,46.80
3054349,1,1,0,13,2,0,100,0,46.80
3054350,1,2,2,13,2,0,100,0,46.80
3054351,1,3,20,13,2,0,100,0,46.80
3054352,1,4,0,13,2,0,100,0,46.80
...,...,...,...,...,...,...,...,...,...
3082855,9,28,1,6,2,0,100,0,47.26
3082856,9,29,0,6,2,0,100,0,47.26
3082857,9,30,1,6,2,0,100,0,47.26
3082858,9,31,9,6,2,0,100,0,47.26


In [183]:
model.fit(x,y)
pred_y = model.predict(test_x)
df = pd.DataFrame(pred_y,columns=['sales'])
df.to_csv(f"{model}.csv",index_label='id')

In [184]:
df

,sales
0,3.312185
1,3.301450
2,3.425946
3,4.632282
4,3.269247
...,...
28507,2.927488
28508,2.849139
28509,2.906019
28510,3.436205


In [185]:
from IPython.display import FileLink

def save_and_download_csv_with_custom_index(df, filename, start_index=3000888):
    # 인덱스를 조정합니다.
    df.index = range(start_index, start_index + len(df))
    # DataFrame을 CSV 파일로 저장합니다.
    df.to_csv(filename)
    # 파일을 다운로드할 수 있는 HTML 링크를 생성하고 표시합니다.
    display(FileLink(filename))

# 예제 DataFrame인 df를 사용하여 파일을 저장하고 다운로드하는 예시:
save_and_download_csv_with_custom_index(df, "submission.csv")


/kaggle/working/submission.csv

# ------------------------------------------------------------------------------------------

In [ ]:
# 월별로 각 상점의 매출
store_nbr_sales_per_month = train.groupby(['date', 'store_nbr']).agg({'sales': 'sum'}).unstack().resample('1M').sum()

# Satışları çiz
store_nbr_sales_per_month.plot(figsize=(21,5))
plt.legend(bbox_to_anchor=(0.9, -0.2), ncol=9)
plt.title("월별 각 상점의 매출")

In [ ]:
# Monthly sales by family
family_sales_per_month = train.groupby(['date', 'family']).agg({'sales': 'sum'}).unstack().resample('1M').sum()

# Satışları çiz
family_sales_per_month.plot(figsize=(21,5))
plt.legend(bbox_to_anchor=(1, -.2), ncol=6)
plt.title("상품별 달단위 매출")

In [ ]:
sns.histplot(data=train, x='onpromotion')

In [ ]:
sns.lineplot(data=oil, x='date', y='dcoilwtico')
plt.title('기름값')

In [ ]:
test.isna().sum()

In [ ]:
holidays_events.columns

In [ ]:
all_df

In [ ]:
holidays_events['description'].value_counts()

In [ ]:
# col = ['date', 'store_nbr', 'family', 'sales', 'onpromotion', 'cluster', 'type_y', 'locale', '', 'dcoilwtico']

In [ ]:
all_df.isna().sum()